In [1]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import argparse
import pandas as pd

torch.cuda.set_device(1)

def cascade(res1,res2,res3,thresh=0.9,verbose=True):
    res1res2 = (res1+res2)/2
    res1res2res3 = (res1+res2+res3)/3
    resFinal = []
    case1 = 0
    case2 = 0
    case3 = 0
    for i,x in enumerate(res1):
        if(x>=thresh):
            resFinal.append(x)
            case1+=1
        else:
            if(res1res2[i]>thresh):
                resFinal.append(res1res2[i])
                case2+=1
            else:
                resFinal.append(res1res2res3[i])
                case3+=1
    if verbose:
        print(case1,case2,case3)
    return resFinal

i = 0

df = pd.read_csv('../trainValid'+str(i)+'.csv')

dfTest = pd.read_csv('../test'+str(i)+'.csv')
dfTest.set = dfTest.set.apply(lambda x : 'test')
dfTest = pd.concat([df[df.set=='validation'],dfTest])

def is_test(path):
    name = path[1]
    return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'

dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
             splitter=FuncSplitter(is_test),
             get_x = ColReader(1,pref="../amd/"),
             get_y=ColReader(3),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)

name1 = "inception_v4"
name2 = "efficientnetv2_rw_s"
name3 = "resnest50d"

learn1 = Learner(dlsTest,timm.create_model(name1,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn1.load(name1+'_512b_'+str(i))

learn2 = Learner(dlsTest,timm.create_model(name2,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn2.load(name2+'_512b_'+str(i))

learn3 = Learner(dlsTest,timm.create_model(name3,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn3.load(name3+'_512b_'+str(i))

res1 = learn1.get_preds()
res2 = learn2.get_preds()
res3 = learn3.get_preds()

res = (res1[0]+res2[0]+res3[0])/3

accuracy = accuracy_score(res1[1],np.argmax(res,axis=1).tolist())
precision = precision_score(res1[1],np.argmax(res,axis=1).tolist())
npv = precision_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
recall = recall_score(res1[1],np.argmax(res,axis=1).tolist())
specificity = recall_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
f1score = f1_score(res1[1],np.argmax(res,axis=1).tolist())
roc = roc_auc_score(res1[1],res[:,1])
cohen = cohen_kappa_score(res1[1],np.argmax(res,axis=1).tolist(),weights='quadratic')

print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.9)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.8)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.7)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])


/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


[0.9174181300427148, 0.925459825750242, 0.909683426443203, 0.9078822412155746, 0.926944971537002, 0.9165867689357623, 0.9695259410629429, 0.8348347347213863]
90 147 1870
[0.9174181300427148, 0.925459825750242, 0.909683426443203, 0.9078822412155746, 0.926944971537002, 0.9165867689357623, 0.9693682638021663, 0.8348347347213863]
502 188 1417
[0.9174181300427148, 0.925459825750242, 0.909683426443203, 0.9078822412155746, 0.926944971537002, 0.9165867689357623, 0.9688348641542821, 0.8348347347213863]
759 110 1238
[0.9174181300427148, 0.925459825750242, 0.909683426443203, 0.9078822412155746, 0.926944971537002, 0.9165867689357623, 0.9675265933962961, 0.8348347347213863]


In [2]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import argparse
import pandas as pd

torch.cuda.set_device(1)

def cascade(res1,res2,res3,thresh=0.9,verbose=True):
    res1res2 = (res1+res2)/2
    res1res2res3 = (res1+res2+res3)/3
    resFinal = []
    case1 = 0
    case2 = 0
    case3 = 0
    for i,x in enumerate(res1):
        if(x>=thresh):
            resFinal.append(x)
            case1+=1
        else:
            if(res1res2[i]>thresh):
                resFinal.append(res1res2[i])
                case2+=1
            else:
                resFinal.append(res1res2res3[i])
                case3+=1
    if verbose:
        print(case1,case2,case3)
    return resFinal

i = 1

df = pd.read_csv('../trainValid'+str(i)+'.csv')

dfTest = pd.read_csv('../test'+str(i)+'.csv')
dfTest.set = dfTest.set.apply(lambda x : 'test')
dfTest = pd.concat([df[df.set=='validation'],dfTest])

def is_test(path):
    name = path[1]
    return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'

dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
             splitter=FuncSplitter(is_test),
             get_x = ColReader(1,pref="../amd/"),
             get_y=ColReader(3),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)


name2 = "inception_v4"
name1 = "efficientnet_b3"
name3 = "resnetrs50"

learn1 = Learner(dlsTest,timm.create_model(name1,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn1.load(name1+'_512b_'+str(i))

learn2 = Learner(dlsTest,timm.create_model(name2,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn2.load(name2+'_512b_'+str(i))

learn3 = Learner(dlsTest,timm.create_model(name3,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn3.load(name3+'_512b_'+str(i))

res1 = learn1.get_preds()
res2 = learn2.get_preds()
res3 = learn3.get_preds()
res = (res1[0]+res2[0]+res3[0])/3

accuracy = accuracy_score(res1[1],np.argmax(res,axis=1).tolist())
precision = precision_score(res1[1],np.argmax(res,axis=1).tolist())
npv = precision_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
recall = recall_score(res1[1],np.argmax(res,axis=1).tolist())
specificity = recall_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
f1score = f1_score(res1[1],np.argmax(res,axis=1).tolist())
roc = roc_auc_score(res1[1],res[:,1])
cohen = cohen_kappa_score(res1[1],np.argmax(res,axis=1).tolist(),weights='quadratic')

print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.9)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.8)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.7)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


[0.9269102990033222, 0.9385964912280702, 0.9158186864014801, 0.9136622390891841, 0.9401709401709402, 0.9259615384615385, 0.9721514927081024, 0.8538224089898331]
428 24 1655
[0.9269102990033222, 0.9385964912280702, 0.9158186864014801, 0.9136622390891841, 0.9401709401709402, 0.9259615384615385, 0.9725402797825314, 0.8538224089898331]
714 37 1356
[0.9269102990033222, 0.9385964912280702, 0.9158186864014801, 0.9136622390891841, 0.9401709401709402, 0.9259615384615385, 0.9729187052083952, 0.8538224089898331]
838 42 1227
[0.9269102990033222, 0.9385964912280702, 0.9158186864014801, 0.9136622390891841, 0.9401709401709402, 0.9259615384615385, 0.9727641814928342, 0.8538224089898331]


In [2]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import argparse
import pandas as pd

torch.cuda.set_device(1)

def cascade(res1,res2,res3,thresh=0.9,verbose=True):
    res1res2 = (res1+res2)/2
    res1res2res3 = (res1+res2+res3)/3
    resFinal = []
    case1 = 0
    case2 = 0
    case3 = 0
    for i,x in enumerate(res1):
        if(x>=thresh):
            resFinal.append(x)
            case1+=1
        else:
            if(res1res2[i]>thresh):
                resFinal.append(res1res2[i])
                case2+=1
            else:
                resFinal.append(res1res2res3[i])
                case3+=1
    if verbose:
        print(case1,case2,case3)
    return resFinal

i = 2

df = pd.read_csv('../trainValid'+str(i)+'.csv')

dfTest = pd.read_csv('../test'+str(i)+'.csv')
dfTest.set = dfTest.set.apply(lambda x : 'test')
dfTest = pd.concat([df[df.set=='validation'],dfTest])

def is_test(path):
    name = path[1]
    return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'

dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
             splitter=FuncSplitter(is_test),
             get_x = ColReader(1,pref="../amd/"),
             get_y=ColReader(3),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)


name3 = "resnest50d"
name2 = "resnetrs50"
name1 = "inception_resnet_v2"

learn3 = Learner(dlsTest,timm.create_model(name3,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn3.load(name3+'_512b_'+str(i))

learn2 = Learner(dlsTest,timm.create_model(name2,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn2.load(name2+'_512b_'+str(i))

learn1 = Learner(dlsTest,timm.create_model(name1,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn1.load(name1+'_512b_'+str(i))

res1 = learn1.get_preds()
res2 = learn2.get_preds()
res3 = learn3.get_preds()
res = (res1[0]+res2[0]+res3[0])/3
accuracy = accuracy_score(res1[1],np.argmax(res,axis=1).tolist())
precision = precision_score(res1[1],np.argmax(res,axis=1).tolist())
npv = precision_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
recall = recall_score(res1[1],np.argmax(res,axis=1).tolist())
specificity = recall_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
f1score = f1_score(res1[1],np.argmax(res,axis=1).tolist())
roc = roc_auc_score(res1[1],res[:,1])
cohen = cohen_kappa_score(res1[1],np.argmax(res,axis=1).tolist(),weights='quadratic')

print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.9)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.8)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.7)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

[0.9021842355175689, 0.9147894221351616, 0.8903225806451613, 0.8869895536562203, 0.9173789173789174, 0.9006750241080038, 0.960782244732862, 0.8043684710351378]
312 78 1716
[0.9021842355175689, 0.9147894221351616, 0.8903225806451613, 0.8869895536562203, 0.9173789173789174, 0.9006750241080038, 0.960880548408247, 0.8043684710351378]
594 79 1433
[0.9021842355175689, 0.9147894221351616, 0.8903225806451613, 0.8869895536562203, 0.9173789173789174, 0.9006750241080038, 0.9599286261204588, 0.8043684710351378]
802 54 1250
[0.9012345679012346, 0.9121951219512195, 0.8908418131359852, 0.8879392212725546, 0.9145299145299145, 0.8999037536092397, 0.9587345521185344, 0.8024691358024691]


In [3]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import argparse
import pandas as pd

torch.cuda.set_device(1)

def cascade(res1,res2,res3,thresh=0.9,verbose=True):
    res1res2 = (res1+res2)/2
    res1res2res3 = (res1+res2+res3)/3
    resFinal = []
    case1 = 0
    case2 = 0
    case3 = 0
    for i,x in enumerate(res1):
        if(x>=thresh):
            resFinal.append(x)
            case1+=1
        else:
            if(res1res2[i]>thresh):
                resFinal.append(res1res2[i])
                case2+=1
            else:
                resFinal.append(res1res2res3[i])
                case3+=1
    if verbose:
        print(case1,case2,case3)
    return resFinal

i = 3

df = pd.read_csv('../trainValid'+str(i)+'.csv')

dfTest = pd.read_csv('../test'+str(i)+'.csv')
dfTest.set = dfTest.set.apply(lambda x : 'test')
dfTest = pd.concat([df[df.set=='validation'],dfTest])

def is_test(path):
    name = path[1]
    return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'

dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
             splitter=FuncSplitter(is_test),
             get_x = ColReader(1,pref="../amd/"),
             get_y=ColReader(3),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)


name1 = "efficientnet_b3"
name2 = "inception_resnet_v2"
name3 = "efficientnetv2_rw_s"

learn1 = Learner(dlsTest,timm.create_model(name1,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn1.load(name1+'_512b_'+str(i))

learn2 = Learner(dlsTest,timm.create_model(name2,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn2.load(name2+'_512b_'+str(i))

learn3 = Learner(dlsTest,timm.create_model(name3,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn3.load(name3+'_512b_'+str(i))

res1 = learn1.get_preds()
res2 = learn2.get_preds()
res3 = learn3.get_preds()
res = (res1[0]+res2[0]+res3[0])/3

accuracy = accuracy_score(res1[1],np.argmax(res,axis=1).tolist())
precision = precision_score(res1[1],np.argmax(res,axis=1).tolist())
npv = precision_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
recall = recall_score(res1[1],np.argmax(res,axis=1).tolist())
specificity = recall_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
f1score = f1_score(res1[1],np.argmax(res,axis=1).tolist())
roc = roc_auc_score(res1[1],res[:,1])
cohen = cohen_kappa_score(res1[1],np.argmax(res,axis=1).tolist(),weights='quadratic')

print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.9)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.8)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.7)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


[0.9349753694581281, 0.9529652351738241, 0.9182509505703422, 0.9155206286836935, 0.9545454545454546, 0.9338677354709419, 0.9789578107891936, 0.8699647493633145]
557 19 1454
[0.9349753694581281, 0.9529652351738241, 0.9182509505703422, 0.9155206286836935, 0.9545454545454546, 0.9338677354709419, 0.9790495391257755, 0.8699647493633145]
752 18 1260
[0.9349753694581281, 0.9529652351738241, 0.9182509505703422, 0.9155206286836935, 0.9545454545454546, 0.9338677354709419, 0.9792058170325446, 0.8699647493633145]
858 15 1157
[0.9354679802955665, 0.9520897043832823, 0.9199237368922784, 0.9174852652259332, 0.9535573122529645, 0.9344672336168083, 0.9791553421806689, 0.8709487376337803]


In [4]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import argparse
import pandas as pd

torch.cuda.set_device(1)

def cascade(res1,res2,res3,thresh=0.9,verbose=True):
    res1res2 = (res1+res2)/2
    res1res2res3 = (res1+res2+res3)/3
    resFinal = []
    case1 = 0
    case2 = 0
    case3 = 0
    for i,x in enumerate(res1):
        if(x>=thresh):
            resFinal.append(x)
            case1+=1
        else:
            if(res1res2[i]>thresh):
                resFinal.append(res1res2[i])
                case2+=1
            else:
                resFinal.append(res1res2res3[i])
                case3+=1
    if verbose:
        print(case1,case2,case3)
    return resFinal

i = 4

df = pd.read_csv('../trainValid'+str(i)+'.csv')

dfTest = pd.read_csv('../test'+str(i)+'.csv')
dfTest.set = dfTest.set.apply(lambda x : 'test')
dfTest = pd.concat([df[df.set=='validation'],dfTest])

def is_test(path):
    name = path[1]
    return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'

dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
             splitter=FuncSplitter(is_test),
             get_x = ColReader(1,pref="../amd/"),
             get_y=ColReader(3),
             item_tfms = [Resize(640)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=512, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)


name1 = "efficientnet_b3"
name2 = "inception_resnet_v2"
name3 = "hrnet_w32"

learn1 = Learner(dlsTest,timm.create_model(name1,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn1.load(name1+'_512b_'+str(i))

learn2 = Learner(dlsTest,timm.create_model(name2,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn2.load(name2+'_512b_'+str(i))

learn3 = Learner(dlsTest,timm.create_model(name3,num_classes=2,pretrained=True,drop_rate=0.5),
            metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
            loss_func= FocalLossFlat()).to_fp16()
learn3.load(name3+'_512b_'+str(i))

res1 = learn1.get_preds()
res2 = learn2.get_preds()
res3 = learn3.get_preds()
res = (res1[0]+res2[0]+res3[0])/3

accuracy = accuracy_score(res1[1],np.argmax(res,axis=1).tolist())
precision = precision_score(res1[1],np.argmax(res,axis=1).tolist())
npv = precision_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
recall = recall_score(res1[1],np.argmax(res,axis=1).tolist())
specificity = recall_score(res1[1],np.argmax(res,axis=1).tolist(),pos_label=0)
f1score = f1_score(res1[1],np.argmax(res,axis=1).tolist())
roc = roc_auc_score(res1[1],res[:,1])
cohen = cohen_kappa_score(res1[1],np.argmax(res,axis=1).tolist(),weights='quadratic')

print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.9)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.8)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

resC = cascade(res1[0][:,1],res2[0][:,1],res3[0][:,1],thresh=0.7)
accuracy = accuracy_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
precision = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
npv = precision_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
recall = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
specificity = recall_score(res1[1],[1 if x >=0.5 else 0 for x in resC],pos_label=0)
f1score = f1_score(res1[1],[1 if x >=0.5 else 0 for x in resC])
roc = roc_auc_score(res1[1],resC)
cohen = cohen_kappa_score(res1[1],[1 if x >=0.5 else 0 for x in resC],weights='quadratic')
print([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


[0.9334326875310481, 0.9485066941297632, 0.9193857965451055, 0.9164179104477612, 0.9503968253968254, 0.9321862348178138, 0.9808408355050146, 0.8668583765470306]
534 22 1457
[0.9334326875310481, 0.9485066941297632, 0.9193857965451055, 0.9164179104477612, 0.9503968253968254, 0.9321862348178138, 0.9812346995182817, 0.8668583765470306]
759 15 1239
[0.9334326875310481, 0.9485066941297632, 0.9193857965451055, 0.9164179104477612, 0.9503968253968254, 0.9321862348178138, 0.9809770591487009, 0.8668583765470306]
865 21 1127
[0.9329359165424739, 0.946611909650924, 0.9201154956689124, 0.9174129353233831, 0.9484126984126984, 0.9317837291561395, 0.9802692884782437, 0.865865378208251]
